In [ ]:

import pandas as pd 
import wandb
import numpy as np

api = wandb.Api()
entity, project = "guneytombak", "aniso_sgdn"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

In [ ]:
def find_params(run):

    data = np.array(run.history()[['Loss', 'EoSq']])
    data = data[~np.isnan(data).any(axis=1)]
    x, y = data[:,0], data[:,1]

    rho = np.corrcoef(x,y)

    # y = a*x**p + b
    # log(y-b) = log(a) + p*log(x)

    b = 0.0
    for k in range(1):
        logA = np.vstack([np.log(x), np.ones(len(x))]).T
        p, _ = np.linalg.lstsq(logA, np.log(y-b/2), rcond=None)[0]
        Ap = np.vstack([x**p, np.ones(len(x))]).T
        a, b = np.linalg.lstsq(Ap, y, rcond=None)[0]
        y_pred = a*(x**p) + b

    return a, b, p, rho[0,1]
        
    
    

In [ ]:
for run in runs: 
    if 'final' in run.TAGS and run.state == 'finished':
        if 'gendata' in run.TAGS:
            # print(run.name[:-9])
            a,b,p,rho= find_params(run)
            print(a,b,p,rho)


In [ ]:
data = np.array(run.history()[['Loss', 'EoSq']])
data = data[~np.isnan(data).any(axis=1)]
data[:,0]